In [1]:
import pandas as pd, numpy as np
from db_queries import get_ids, get_outputs, get_location_metadata, get_population, get_covariate_estimates
from get_draws.api import get_draws
import scipy.stats as sp
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import random

In [13]:
location_ids = [168, 161, 201, 202, 6, 205, 171, 141, 179, 207, 163, 11, 180, 181,
       184, 15, 164, 213, 214, 165, 196, 522, 190, 189, 20]

In [14]:
year_id = 2019
age_group_ids = [2,3,4,5]

In [16]:
seqs = [1004, 1005, 1006, 1008, 1009, 1010, 1012, 1013, 1014, 1016, 1017, 1018, 1020, 1021, 1022, 1024, 1025, 1026, 1028, 1029, 1030, 1032, 1033, 1034, 1361, 1364, 1367, 1373, 1376, 1379, 1385, 1388, 1391, 1397, 1400, 1403, 1409, 1412, 1415, 1421, 1424, 1427, 1433, 1436, 1439, 1445, 1448, 1451, 5213, 5216, 5219, 5222, 5225, 5228, 5237, 5240, 5243, 5246, 5249, 5252, 5261, 5264, 5267, 5270, 5273, 5276,
  4985, 4988, 4991, 4994, 4997, 5000, 5009, 5012, 5015, 5678, 5681, 5684, 7214, 7217, 7220,
        4952, 4955, 4958, 4961, 4964, 4967, 4976, 4979, 4982, 5627, 5630, 5633, 7202, 7205, 7208,
        5393, 5396, 5399, 182, 183, 184, 240, 241, 242, 177, 178, 179, 144,145,146,172,173,174,525,526,527,1106,1107,1108,537,538,539,206,207,208,
        22989, 22990, 22991, 22992, 22993, 22999, 23000, 23001, 23002,
       23003, 23009, 23010, 23011, 23012, 23013,
       5567, 5570, 5573, 5579, 5582, 5585,
       23030, 23031, 23032, 23034, 23035, 23036, 23038, 23039, 23040,
       23042, 23043, 23044, 23046, 23047, 23048]
    

In [48]:
s = get_ids('sequela')
pd.DataFrame.to_csv(s[s['sequela_id'].isin(seqs)], 'out.csv')

# Under five, selected locations

In [55]:
ylds = get_draws('sequela_id', seqs, 
                 source='como',
                 location_id=location_ids, 
                 age_group_id=age_group_ids,
                 year_id=2019,
                 measure_id=3,
                 decomp_step='step5',
                 gbd_round_id=6)
ylds.head()

,age_group_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,draw_105,...,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,sequela_id,sex_id,year_id,metric_id
0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11,3,144,1,2019,3
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11,3,145,1,2019,3
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11,3,146,1,2019,3
3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11,3,172,1,2019,3
4,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11,3,173,1,2019,3


In [56]:
# sum over causes
yld_sum = ylds.groupby(['location_id','sex_id','age_group_id']).sum()
yld_sum = yld_sum.drop(columns=[c for c in yld_sum.columns if 'draw' not in c])
yld_sum.head()

draw_0    draw_1   draw_10  draw_100  \
location_id sex_id age_group_id                                           
6           1      2             0.000254  0.000235  0.000140  0.000191   
                   3             0.006555  0.006429  0.003696  0.004833   
                   4             0.001691  0.002696  0.001853  0.002878   
                   5             0.000892  0.001146  0.000641  0.001297   
            2      2             0.000028  0.000005  0.000009  0.000015   

                                 draw_101  draw_102  draw_103  draw_104  \
location_id sex_id age_group_id                                           
6           1      2             0.000125  0.000253  0.000257  0.000218   
                   3             0.004512  0.005968  0.005664  0.006497   
                   4             0.001583  0.001936  0.002799  0.002545   
                   5             0.000795  0.001220  0.001321  0.001372   
            2      2             0.000013  0.000035  0.000022  0.000008   

                                 draw_105  draw_106  ...  draw_990  draw_991  \
location_id sex_id age_group_id                      ...                       
6           1      2             0.000115  0.000117  ...  0.000199  0.000167   
                   3             0.007855  0.003064  ...  0.003961  0.005031   
                   4             0.003191  0.002281  ...  0.001755  0.002769   
                   5             0.001897  0.000900  ...  0.001486  0.001333   
            2      2             0.000050  0.000007  ...  0.000011  0.000032   

                                 draw_992  draw_993  draw_994  draw_995  \
location_id sex_id age_group_id                                           
6           1      2             0.000353  0.000230  0.000115  0.000153   
                   3             0.005768  0.006013  0.004859  0.005139   
                   4             0.001671  0.001426  0.002362  0.003049   
                   5             0.001277  0.001323  0.000750  0.001085   
            2      2             0.000008  0.000012  0.000019  0.000010   

                                 draw_996  draw_997  draw_998  draw_999  
location_id sex_id age_group_id                                          
6           1      2             0.000141  0.000063  0.000187  0.000102  
                   3             0.002383  0.005167  0.005484  0.003906  
                   4             0.000898  0.001525  0.001450  0.001724  
                   5             0.000590  0.001508  0.000564  0.001044  
            2      2             0.000008  0.000006  0.000013  0.000013  

[5 rows x 1000 columns]

In [57]:
pop = get_population(location_id=location_ids,
                     sex_id=[1,2],
                    gbd_round_id=6,
                    decomp_step='step4',
                    age_group_id=age_group_ids)
pop_sum = pop.groupby(['location_id','sex_id','age_group_id']).sum().drop(columns=['year_id','run_id'])
pop_sum.head()

population
location_id sex_id age_group_id              
6           1      2             1.532274e+05
                   3             4.594606e+05
                   4             7.503016e+06
                   5             3.575678e+07
            2      2             1.311718e+05

In [58]:
yld_scaled = pd.DataFrame((pop_sum.values * yld_sum).sum()).reset_index()
yld_scaled.head()

,index,0
0,draw_0,5.822666e+06
1,draw_1,5.585315e+06
2,draw_10,4.471592e+06
3,draw_100,5.592735e+06
4,draw_101,4.480395e+06


In [59]:
yld_scaled = pd.DataFrame((pop_sum.values * yld_sum).sum()).reset_index()
yld_scaled_mean = yld_scaled[0].mean()
yld_scaled_lower = yld_scaled[0].quantile(0.025)
yld_scaled_upper = yld_scaled[0].quantile(0.975)

print('YLDs due to iron responsive anemia - children under 5 in selected countries.',
    'mean:', yld_scaled_mean.round(1), 
      'lower:', yld_scaled_lower.round(1), 
      'upper:', yld_scaled_upper.round(1))

YLDs due to iron responsive anemia - children under 5 in selected countries. mean: 5043802.1 lower: 3363198.7 upper: 7324331.8


# Under 5 globally

In [52]:
ylds = get_draws('sequela_id', seqs, 
                 source='como',
                 location_id=1, 
                 age_group_id=age_group_ids,
                 year_id=2019,
                 measure_id=3, 
                 decomp_step='step5',
                 gbd_round_id=6)
# sum over causes
yld_sum = ylds.groupby(['sex_id','age_group_id']).sum()
yld_sum = yld_sum.drop(columns=[c for c in yld_sum.columns if 'draw' not in c])
yld_sum.head()

draw_0    draw_1   draw_10  draw_100  draw_101  \
sex_id age_group_id                                                     
1      2             0.003992  0.003677  0.002815  0.003291  0.002532   
       3             0.023294  0.022104  0.017402  0.022141  0.018155   
       4             0.013691  0.014288  0.011304  0.013703  0.009904   
       5             0.011890  0.011221  0.008784  0.011076  0.009166   
2      2             0.002033  0.001947  0.001542  0.001914  0.001375   

                     draw_102  draw_103  draw_104  draw_105  draw_106  ...  \
sex_id age_group_id                                                    ...   
1      2             0.003511  0.003493  0.002765  0.004411  0.002012  ...   
       3             0.020508  0.023382  0.017880  0.024219  0.014057  ...   
       4             0.013039  0.013964  0.011636  0.015079  0.009430  ...   
       5             0.010198  0.010875  0.008423  0.012753  0.007135  ...   
2      2             0.001825  0.002064  0.001659  0.002550  0.001170  ...   

                     draw_990  draw_991  draw_992  draw_993  draw_994  \
sex_id age_group_id                                                     
1      2             0.002160  0.003404  0.002921  0.002811  0.002758   
       3             0.017616  0.021179  0.020422  0.018417  0.018546   
       4             0.011478  0.013181  0.012254  0.011801  0.011901   
       5             0.009354  0.011085  0.009913  0.009383  0.009495   
2      2             0.001485  0.001872  0.001807  0.001766  0.001765   

                     draw_995  draw_996  draw_997  draw_998  draw_999  
sex_id age_group_id                                                    
1      2             0.003320  0.001557  0.002465  0.002540  0.002312  
       3             0.020763  0.009605  0.016105  0.016702  0.015815  
       4             0.012714  0.006323  0.009873  0.010061  0.010332  
       5             0.010651  0.004994  0.008193  0.008098  0.007784  
2      2             0.001860  0.000849  0.001511  0.001475  0.001320  

[5 rows x 1000 columns]

In [53]:
pop = get_population(location_id=1,
                     sex_id=[1,2],
                    gbd_round_id=6,
                    decomp_step='step4',
                    age_group_id=age_group_ids)
pop_sum = pop.groupby(['sex_id','age_group_id']).sum().drop(columns=['location_id','year_id','run_id'])
pop_sum.head()

population
sex_id age_group_id              
1      2             1.330245e+06
       3             3.951181e+06
       4             6.292360e+07
       5             2.741937e+08
2      2             1.241845e+06

In [54]:
yld_scaled = pd.DataFrame((pop_sum.values * yld_sum).sum()).reset_index()
yld_scaled_mean = yld_scaled[0].mean()
yld_scaled_lower = yld_scaled[0].quantile(0.025)
yld_scaled_upper = yld_scaled[0].quantile(0.975)

print('YLDs due to iron responsive anemia - children under 5 globally.',
    'mean:', yld_scaled_mean.round(1), 
      'lower:', yld_scaled_lower.round(1), 
      'upper:', yld_scaled_upper.round(1))

YLDs due to iron responsive anemia - children under 5 globally. mean: 6830530.9 lower: 4567380.6 upper: 9887678.6


# Total population, selected locations

In [11]:
ages = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 30, 31, 32, 235]

In [17]:
ylds = get_draws('sequela_id', seqs, 
                 source='como',
                 location_id=location_ids, 
                 age_group_id=ages,
                 year_id=2019,
                 measure_id=3, 
                 decomp_step='step5',
                 gbd_round_id=6)
ylds.head()

,age_group_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,draw_105,...,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,sequela_id,sex_id,year_id,metric_id
0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11,3,144,1,2019,3
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11,3,145,1,2019,3
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11,3,146,1,2019,3
3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11,3,172,1,2019,3
4,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11,3,173,1,2019,3


In [18]:
# sum over causes
yld_sum = ylds.groupby(['location_id','sex_id','age_group_id']).sum()
yld_sum = yld_sum.drop(columns=[c for c in yld_sum.columns if 'draw' not in c])
yld_sum.head()

draw_0    draw_1   draw_10  draw_100  \
location_id sex_id age_group_id                                           
6           1      2             0.000254  0.000235  0.000140  0.000191   
                   3             0.006555  0.006429  0.003696  0.004833   
                   4             0.001691  0.002696  0.001853  0.002878   
                   5             0.000892  0.001146  0.000641  0.001297   
                   6             0.001587  0.001718  0.000454  0.000956   

                                 draw_101  draw_102  draw_103  draw_104  \
location_id sex_id age_group_id                                           
6           1      2             0.000125  0.000253  0.000257  0.000218   
                   3             0.004512  0.005968  0.005664  0.006497   
                   4             0.001583  0.001936  0.002799  0.002545   
                   5             0.000795  0.001220  0.001321  0.001372   
                   6             0.001050  0.000791  0.001214  0.000334   

                                 draw_105  draw_106  ...  draw_990  draw_991  \
location_id sex_id age_group_id                      ...                       
6           1      2             0.000115  0.000117  ...  0.000199  0.000167   
                   3             0.007855  0.003064  ...  0.003961  0.005031   
                   4             0.003191  0.002281  ...  0.001755  0.002769   
                   5             0.001897  0.000900  ...  0.001486  0.001333   
                   6             0.001198  0.000610  ...  0.000704  0.001538   

                                 draw_992  draw_993  draw_994  draw_995  \
location_id sex_id age_group_id                                           
6           1      2             0.000353  0.000230  0.000115  0.000153   
                   3             0.005768  0.006013  0.004859  0.005139   
                   4             0.001671  0.001426  0.002362  0.003049   
                   5             0.001277  0.001323  0.000750  0.001085   
                   6             0.001202  0.000441  0.000815  0.001078   

                                 draw_996  draw_997  draw_998  draw_999  
location_id sex_id age_group_id                                          
6           1      2             0.000141  0.000063  0.000187  0.000102  
                   3             0.002383  0.005167  0.005484  0.003906  
                   4             0.000898  0.001525  0.001450  0.001724  
                   5             0.000590  0.001508  0.000564  0.001044  
                   6             0.000490  0.001188  0.001012  0.000787  

[5 rows x 1000 columns]

In [23]:
pop = get_population(location_id=location_ids,
                     sex_id=[1,2],
                    gbd_round_id=6,
                    decomp_step='step4',
                    age_group_id=ages)
pop_sum = pop.groupby(['location_id','sex_id','age_group_id']).sum().drop(columns=['year_id','run_id'])
pop_sum.head()

population
location_id sex_id age_group_id              
6           1      2             1.532274e+05
                   3             4.594606e+05
                   4             7.503016e+06
                   5             3.575678e+07
                   6             3.926615e+07

In [24]:
yld_scaled = pd.DataFrame((pop_sum.values * yld_sum).sum()).reset_index()
yld_scaled_mean = yld_scaled[0].mean()
yld_scaled_lower = yld_scaled[0].quantile(0.025)
yld_scaled_upper = yld_scaled[0].quantile(0.975)

print('YLDs due to iron responsive anemia - total population in selected locations',
    'mean:', yld_scaled_mean.round(1), 
      'lower:', yld_scaled_lower.round(1), 
      'upper:', yld_scaled_upper.round(1))

YLDs due to iron responsive anemia - total population in selected locations mean: 32350391.7 lower: 22246641.9 upper: 46058167.8


# Total population, globally

In [26]:
ylds = get_draws('sequela_id', seqs, 
                 source='como',
                 location_id=1, 
                 age_group_id=ages,
                 year_id=2019,
                 measure_id=3, 
                 decomp_step='step5',
                 gbd_round_id=6)
# sum over causes, locations
yld_sum = ylds.groupby(['sex_id','age_group_id']).sum()
yld_sum = yld_sum.drop(columns=[c for c in yld_sum.columns if 'draw' not in c])
yld_sum.head()

draw_0    draw_1   draw_10  draw_100  draw_101  \
sex_id age_group_id                                                     
1      2             0.003992  0.003677  0.002815  0.003291  0.002532   
       3             0.023294  0.022104  0.017402  0.022141  0.018155   
       4             0.013691  0.014288  0.011304  0.013703  0.009904   
       5             0.011890  0.011221  0.008784  0.011076  0.009166   
       6             0.012567  0.011923  0.009952  0.012616  0.010318   

                     draw_102  draw_103  draw_104  draw_105  draw_106  ...  \
sex_id age_group_id                                                    ...   
1      2             0.003511  0.003493  0.002765  0.004411  0.002012  ...   
       3             0.020508  0.023382  0.017880  0.024219  0.014057  ...   
       4             0.013039  0.013964  0.011636  0.015079  0.009430  ...   
       5             0.010198  0.010875  0.008423  0.012753  0.007135  ...   
       6             0.011716  0.012307  0.008986  0.012615  0.007675  ...   

                     draw_990  draw_991  draw_992  draw_993  draw_994  \
sex_id age_group_id                                                     
1      2             0.002160  0.003404  0.002921  0.002811  0.002758   
       3             0.017616  0.021179  0.020422  0.018417  0.018546   
       4             0.011478  0.013181  0.012254  0.011801  0.011901   
       5             0.009354  0.011085  0.009913  0.009383  0.009495   
       6             0.009971  0.010962  0.010928  0.009896  0.009721   

                     draw_995  draw_996  draw_997  draw_998  draw_999  
sex_id age_group_id                                                    
1      2             0.003320  0.001557  0.002465  0.002540  0.002312  
       3             0.020763  0.009605  0.016105  0.016702  0.015815  
       4             0.012714  0.006323  0.009873  0.010061  0.010332  
       5             0.010651  0.004994  0.008193  0.008098  0.007784  
       6             0.011174  0.005626  0.009593  0.008807  0.008338  

[5 rows x 1000 columns]

In [27]:
pop = get_population(location_id=1,
                     sex_id=[1,2],
                    gbd_round_id=6,
                    decomp_step='step4',
                    age_group_id=ages)
pop_sum = pop.groupby(['sex_id','age_group_id']).sum().drop(columns=['location_id','year_id','run_id'])
pop_sum.head()

population
sex_id age_group_id              
1      2             1.330245e+06
       3             3.951181e+06
       4             6.292360e+07
       5             2.741937e+08
       6             3.379492e+08

In [28]:
yld_scaled = pd.DataFrame((pop_sum.values * yld_sum).reset_index().sum()).reset_index()
yld_scaled = yld_scaled.loc[yld_scaled['index'] != 'sex_id'].loc[yld_scaled['index'] != 'age_group_id']
yld_scaled_mean = yld_scaled[0].mean()
yld_scaled_lower = yld_scaled[0].quantile(0.025)
yld_scaled_upper = yld_scaled[0].quantile(0.975)

print('YLDs due to iron responsive anemia - total population globally.',
    'mean:', yld_scaled_mean.round(1), 
      'lower:', yld_scaled_lower.round(1), 
      'upper:', yld_scaled_upper.round(1))

YLDs due to iron responsive anemia - total population globally. mean: 43564094.5 lower: 30082889.1 upper: 61507339.1


# Women of reproductive age in selected locations

In [34]:
ages = [7, 8, 9, 10, 11, 12, 13, 14, 15] # 10-54

In [35]:
ylds = get_draws('sequela_id', seqs, 
                 source='como',
                 location_id=location_ids, 
                 age_group_id=ages,
                 year_id=2019,
                 measure_id=3, 
                 sex_id=2,
                 decomp_step='step5',
                 gbd_round_id=6)
# sum over causes, locations
yld_sum = ylds.groupby(['location_id','sex_id','age_group_id']).sum()
yld_sum = yld_sum.drop(columns=[c for c in yld_sum.columns if 'draw' not in c])
yld_sum.head()

draw_0    draw_1   draw_10  draw_100  \
location_id sex_id age_group_id                                           
6           2      7             0.000658  0.000493  0.000814  0.000792   
                   8             0.001246  0.001092  0.000580  0.000718   
                   9             0.001122  0.001297  0.001000  0.001344   
                   10            0.001471  0.001124  0.000770  0.001190   
                   11            0.001654  0.001699  0.000999  0.002312   

                                 draw_101  draw_102  draw_103  draw_104  \
location_id sex_id age_group_id                                           
6           2      7             0.000404  0.000564  0.001260  0.000613   
                   8             0.001229  0.001373  0.001139  0.001555   
                   9             0.001611  0.001324  0.001245  0.000833   
                   10            0.001467  0.000970  0.001519  0.001595   
                   11            0.001830  0.001646  0.001765  0.002515   

                                 draw_105  draw_106  ...  draw_990  draw_991  \
location_id sex_id age_group_id                      ...                       
6           2      7             0.000309  0.000500  ...  0.000597  0.000347   
                   8             0.001279  0.000713  ...  0.002062  0.001649   
                   9             0.001356  0.000965  ...  0.000830  0.001053   
                   10            0.002177  0.000910  ...  0.000937  0.001449   
                   11            0.002293  0.001051  ...  0.001443  0.001706   

                                 draw_992  draw_993  draw_994  draw_995  \
location_id sex_id age_group_id                                           
6           2      7             0.000536  0.000443  0.000253  0.001393   
                   8             0.001802  0.000893  0.001346  0.000932   
                   9             0.001311  0.001356  0.001776  0.000619   
                   10            0.001858  0.001438  0.001341  0.001055   
                   11            0.001863  0.002027  0.002145  0.000799   

                                 draw_996  draw_997  draw_998  draw_999  
location_id sex_id age_group_id                                          
6           2      7             0.000325  0.000360  0.000286  0.000476  
                   8             0.000421  0.000847  0.000394  0.000930  
                   9             0.000741  0.001055  0.001156  0.000802  
                   10            0.000850  0.001181  0.001094  0.001625  
                   11            0.000661  0.001557  0.001873  0.001983  

[5 rows x 1000 columns]

In [39]:
pop = get_population(location_id=location_ids,
                     sex_id=2,
                    gbd_round_id=6,
                    decomp_step='step4',
                    age_group_id=ages)
pop_sum = pop.groupby(['location_id','sex_id','age_group_id']).sum().drop(columns=['year_id','run_id'])
pop_sum.head()

population
location_id sex_id age_group_id              
6           2      7             3.233294e+07
                   8             3.504047e+07
                   9             3.910040e+07
                   10            5.436141e+07
                   11            6.380212e+07

In [40]:
yld_scaled = pd.DataFrame((pop_sum.values * yld_sum).reset_index().sum()).reset_index()
yld_scaled = yld_scaled.loc[yld_scaled['index'] != 'sex_id'].loc[yld_scaled['index'] != 'age_group_id']
yld_scaled_mean = yld_scaled[0].mean()
yld_scaled_lower = yld_scaled[0].quantile(0.025)
yld_scaled_upper = yld_scaled[0].quantile(0.975)

print('YLDs due to iron responsive anemia - women of reproductive age in selected locations.',
    'mean:', yld_scaled_mean.round(1), 
      'lower:', yld_scaled_lower.round(1), 
      'upper:', yld_scaled_upper.round(1))

YLDs due to iron responsive anemia - women of reproductive age in selected locations. mean: 11902393.5 lower: 8204885.9 upper: 16772010.1


# Women of reproductive age globally

In [41]:
ylds = get_draws('sequela_id', seqs, 
                 source='como',
                 location_id=1, 
                 age_group_id=ages,
                 year_id=2019,
                 measure_id=3, 
                 sex_id=2,
                 decomp_step='step5',
                 gbd_round_id=6)
# sum over causes, locations
yld_sum = ylds.groupby(['location_id','sex_id','age_group_id']).sum()
yld_sum = yld_sum.drop(columns=[c for c in yld_sum.columns if 'draw' not in c])
yld_sum.head()

draw_0    draw_1   draw_10  draw_100  \
location_id sex_id age_group_id                                           
1           2      7             0.008585  0.007903  0.006434  0.007520   
                   8             0.008251  0.006903  0.006196  0.007498   
                   9             0.006979  0.006608  0.005435  0.006785   
                   10            0.006299  0.005907  0.005072  0.006231   
                   11            0.006780  0.006546  0.004880  0.006748   

                                 draw_101  draw_102  draw_103  draw_104  \
location_id sex_id age_group_id                                           
1           2      7             0.006713  0.007743  0.007868  0.006490   
                   8             0.006216  0.007072  0.007450  0.005932   
                   9             0.005331  0.006047  0.007084  0.005270   
                   10            0.005052  0.005573  0.006551  0.005040   
                   11            0.005349  0.006068  0.006516  0.005518   

                                 draw_105  draw_106  ...  draw_990  draw_991  \
location_id sex_id age_group_id                      ...                       
1           2      7             0.008356  0.004714  ...  0.006268  0.008440   
                   8             0.008533  0.004696  ...  0.005816  0.007607   
                   9             0.006903  0.004463  ...  0.005282  0.006174   
                   10            0.006320  0.004347  ...  0.004712  0.006092   
                   11            0.006840  0.004623  ...  0.005322  0.006371   

                                 draw_992  draw_993  draw_994  draw_995  \
location_id sex_id age_group_id                                           
1           2      7             0.007127  0.007079  0.006797  0.008402   
                   8             0.007302  0.006516  0.006692  0.007272   
                   9             0.006010  0.005518  0.006078  0.006147   
                   10            0.005793  0.004876  0.005591  0.005584   
                   11            0.006344  0.005654  0.005717  0.005973   

                                 draw_996  draw_997  draw_998  draw_999  
location_id sex_id age_group_id                                          
1           2      7             0.003705  0.005678  0.005986  0.005899  
                   8             0.003401  0.005331  0.005476  0.005628  
                   9             0.003117  0.004829  0.005090  0.005003  
                   10            0.002992  0.004583  0.004615  0.004757  
                   11            0.003102  0.004925  0.005029  0.005146  

[5 rows x 1000 columns]

In [42]:
pop = get_population(location_id=1,
                     sex_id=2,
                    gbd_round_id=6,
                    decomp_step='step4',
                    age_group_id=ages)
pop_sum = pop.groupby(['location_id','sex_id','age_group_id']).sum().drop(columns=['year_id','run_id'])
pop_sum.head()

population
location_id sex_id age_group_id              
1           2      7             3.108525e+08
                   8             3.017589e+08
                   9             2.957762e+08
                   10            3.006931e+08
                   11            2.985578e+08

In [43]:
yld_scaled = pd.DataFrame((pop_sum.values * yld_sum).reset_index().sum()).reset_index()
yld_scaled = yld_scaled.loc[yld_scaled['index'] != 'sex_id'].loc[yld_scaled['index'] != 'age_group_id']
yld_scaled_mean = yld_scaled[0].mean()
yld_scaled_lower = yld_scaled[0].quantile(0.025)
yld_scaled_upper = yld_scaled[0].quantile(0.975)

print('YLDs due to iron responsive anemia - women of reproductive age globally.',
    'mean:', yld_scaled_mean.round(1), 
      'lower:', yld_scaled_lower.round(1), 
      'upper:', yld_scaled_upper.round(1))

YLDs due to iron responsive anemia - women of reproductive age globally. mean: 16046490.3 lower: 11062340.2 upper: 22501561.9
